In [103]:
import pandas as pd
import numpy as np

from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
import scipy.cluster.hierarchy as shc
import matplotlib.pyplot as plt
from sklearn.cluster import AgglomerativeClustering
import statsmodels.api as sm
from sklearn.metrics import classification_report
import seaborn as sns

import plotly.express as px
import plotly.graph_objects as go

import warnings
warnings.filterwarnings("ignore")

In [104]:
df_elasticidad = pd.read_pickle("../output/base_etapa_3.pkl")

In [105]:
df_elasticidad

,irr,prob_seg0,prob_seg1,prob_seg2,prob_seg3,prob_seg4,requested_amount,emi_amount,cibil_score,loan_term,...,coef_seg0,coef_seg1,coef_seg2,coef_seg3,coef_seg4,elasticidad_seg0,elasticidad_seg1,elasticidad_seg2,elasticidad_seg3,elasticidad_seg4
0,16.0000,0.935652,0.939023,0.908777,0.914462,0.886694,300000.0,9626.0,733.0,36.0,...,-17.624722,1.969908,3.172451,1.340623,3.926671,-3.144440,-2.646663,-3.655322,-3.861949,-4.303271
1,16.2105,0.920310,0.926200,0.891861,0.896278,0.867423,300000.0,9626.0,733.0,36.0,...,-17.624722,1.969908,3.172451,1.340623,3.926671,-3.912473,-3.218349,-4.353569,-4.705021,-5.058883
2,16.4211,0.901949,0.911142,0.872511,0.875048,0.845737,480000.0,13739.0,773.0,48.0,...,-17.624722,1.969908,3.172451,1.340623,3.926671,-4.836244,-3.892965,-5.156368,-5.694349,-5.913652
3,16.6316,0.880227,0.893619,0.850593,0.850551,0.821578,450000.0,13611.0,778.0,48.0,...,-17.624722,1.969908,3.172451,1.340623,3.926671,-5.934567,-4.681858,-6.070353,-6.841716,-6.870935
4,16.8421,0.854819,0.873400,0.825993,0.822608,0.794912,446500.0,13528.0,774.0,48.0,...,-17.624722,1.969908,3.172451,1.340623,3.926671,-7.225656,-5.596641,-7.101473,-8.157247,-7.933162
5,17.0526,0.825490,0.850295,0.798667,0.791147,0.765782,400000.0,12500.0,775.0,36.0,...,-17.624722,1.969908,3.172451,1.340623,3.926671,-8.723585,-6.647181,-8.252858,-9.646201,-9.099785
6,17.2632,0.792105,0.824156,0.768630,0.756204,0.734293,426000.0,13100.0,774.0,36.0,...,-17.624722,1.969908,3.172451,1.340623,3.926671,-10.437430,-7.841584,-9.525153,-11.308858,-10.367893
7,17.4737,0.754740,0.794950,0.736029,0.718011,0.700671,400000.0,12300.0,774.0,36.0,...,-17.624722,1.969908,3.172451,1.340623,3.926671,-12.365761,-9.182874,-10.913504,-13.136133,-11.729510
8,17.6842,0.713614,0.762705,0.701064,0.676913,0.665180,384000.0,12350.5,771.0,36.0,...,-17.624722,1.969908,3.172451,1.340623,3.926671,-14.499712,-10.671434,-12.410825,-15.113652,-13.175176
9,17.8947,0.669174,0.727569,0.664041,0.633423,0.628170,333000.0,10810.0,753.0,36.0,...,-17.624722,1.969908,3.172451,1.340623,3.926671,-16.818739,-12.302002,-14.005337,-17.218683,-14.691775


In [106]:
df_elasticidad['costo'] = 12
df_elasticidad["factor_ingreso"] = (df_elasticidad["irr"]  / 12) / (1-(1+(df_elasticidad["irr"]/12))**(-df_elasticidad["loan_term"]))
df_elasticidad["factor_gasto"] = (df_elasticidad["costo"]  / 12) / (1-(1+(df_elasticidad["costo"]/12))**(-df_elasticidad["loan_term"]))


In [107]:
df_elasticidad['ingresos_seg0'] = df_elasticidad['prob_seg0'] * 1000 * 36 * df_elasticidad["factor_ingreso"]
df_elasticidad['ingresos_seg1'] = df_elasticidad['prob_seg1'] * 1000 * 36 * df_elasticidad["factor_ingreso"]
df_elasticidad['ingresos_seg2'] = df_elasticidad['prob_seg2'] * 1000 * 36 * df_elasticidad["factor_ingreso"]
df_elasticidad['ingresos_seg3'] = df_elasticidad['prob_seg3'] * 1000 * 36 * df_elasticidad["factor_ingreso"]
df_elasticidad['ingresos_seg4'] = df_elasticidad['prob_seg4'] * 1000 * 36 * df_elasticidad["factor_ingreso"]

df_elasticidad['gastos_seg0'] = df_elasticidad['prob_seg0'] * 1000 * 36 * df_elasticidad["factor_gasto"]
df_elasticidad['gastos_seg1'] = df_elasticidad['prob_seg1'] * 1000 * 36 * df_elasticidad["factor_gasto"]
df_elasticidad['gastos_seg2'] = df_elasticidad['prob_seg2'] * 1000 * 36 * df_elasticidad["factor_gasto"]
df_elasticidad['gastos_seg3'] = df_elasticidad['prob_seg3'] * 1000 * 36 * df_elasticidad["factor_gasto"]
df_elasticidad['gastos_seg4'] = df_elasticidad['prob_seg4'] * 1000 * 36 * df_elasticidad["factor_gasto"]

df_elasticidad['rentabilidad_seg0'] = df_elasticidad['ingresos_seg0'] - df_elasticidad["gastos_seg0"]
df_elasticidad['rentabilidad_seg1'] = df_elasticidad['ingresos_seg1'] - df_elasticidad["gastos_seg1"]
df_elasticidad['rentabilidad_seg2'] = df_elasticidad['ingresos_seg2'] - df_elasticidad["gastos_seg2"]
df_elasticidad['rentabilidad_seg3'] = df_elasticidad['ingresos_seg3'] - df_elasticidad["gastos_seg3"]
df_elasticidad['rentabilidad_seg4'] = df_elasticidad['ingresos_seg4'] - df_elasticidad["gastos_seg4"]

In [108]:
# Graficamos las curvas de Precio-Ingresos
fig = px.line(df_elasticidad, x='irr', y=['ingresos_seg0', 'ingresos_seg1',
                                             'ingresos_seg2', 'ingresos_seg3', 'ingresos_seg4'
                                             ],
              labels={'irr':'tasa', 'value':'Ingresos'},
              title='Curva de Ingresos',
              width=800, height=600)

fig.show()

In [109]:
# Graficamos las curvas de Precio-Rentabilidad
fig = px.line(df_elasticidad, x='irr', y=['rentabilidad_seg0',
                                             'rentabilidad_seg1',
                                             'rentabilidad_seg2',
                                             'rentabilidad_seg3',
                                             'rentabilidad_seg4',
                                             ],
              labels={'irr':'tasa', 'value':'Rentabilidad'},
              title='Curvas de Rentabilidad',
              width=800, height=600)

fig.show()

In [110]:
df_elasticidad.head()

,irr,prob_seg0,prob_seg1,prob_seg2,prob_seg3,prob_seg4,requested_amount,emi_amount,cibil_score,loan_term,...,gastos_seg0,gastos_seg1,gastos_seg2,gastos_seg3,gastos_seg4,rentabilidad_seg0,rentabilidad_seg1,rentabilidad_seg2,rentabilidad_seg3,rentabilidad_seg4
0,16.0000,0.935652,0.939023,0.908777,0.914462,0.886694,300000.0,9626.0,733.0,36.0,...,33683.469155,33804.831709,32715.973702,32920.644493,31920.969648,11227.823051,11268.277236,10905.324567,10973.548164,10640.323215
1,16.2105,0.920310,0.926200,0.891861,0.896278,0.867423,300000.0,9626.0,733.0,36.0,...,33131.177924,33343.195006,32107.004994,32266.016723,31227.231756,11624.902054,11699.293547,11265.545377,11321.338617,10956.854942
2,16.4211,0.901949,0.911142,0.872511,0.875048,0.845737,480000.0,13739.0,773.0,48.0,...,32470.178795,32801.109921,31410.402456,31501.713000,30446.538101,11962.825622,12084.748922,11572.377525,11606.018612,11217.265800
3,16.6316,0.880227,0.893619,0.850593,0.850551,0.821578,450000.0,13611.0,778.0,48.0,...,31688.172261,32170.298350,30621.360349,30619.830949,29576.796374,12230.578220,12416.662820,11818.824383,11818.234085,11415.657507
4,16.8421,0.854819,0.873400,0.825993,0.822608,0.794912,446500.0,13528.0,774.0,48.0,...,30773.499088,31442.407129,29735.760389,29613.897637,28616.818536,12417.363328,12687.273296,11998.627115,11949.454479,11547.124753


In [111]:
# Hallamos los precios que maximicen los ingresos en cada segmento
max_ing_seg0 = df_elasticidad.loc[(df_elasticidad['ingresos_seg0'] == df_elasticidad['ingresos_seg0'].max())]
max_ing_seg1 = df_elasticidad.loc[(df_elasticidad['ingresos_seg1'] == df_elasticidad['ingresos_seg1'].max())]
max_ing_seg2 = df_elasticidad.loc[(df_elasticidad['ingresos_seg2'] == df_elasticidad['ingresos_seg2'].max())]
max_ing_seg3 = df_elasticidad.loc[(df_elasticidad['ingresos_seg3'] == df_elasticidad['ingresos_seg3'].max())]
max_ing_seg4 = df_elasticidad.loc[(df_elasticidad['ingresos_seg4'] == df_elasticidad['ingresos_seg4'].max())]

In [112]:
max_ing_seg0

,irr,prob_seg0,prob_seg1,prob_seg2,prob_seg3,prob_seg4,requested_amount,emi_amount,cibil_score,loan_term,...,gastos_seg0,gastos_seg1,gastos_seg2,gastos_seg3,gastos_seg4,rentabilidad_seg0,rentabilidad_seg1,rentabilidad_seg2,rentabilidad_seg3,rentabilidad_seg4
0,16.0,0.935652,0.939023,0.908777,0.914462,0.886694,300000.0,9626.0,733.0,36.0,...,33683.469155,33804.831709,32715.973702,32920.644493,31920.969648,11227.823051,11268.277236,10905.324567,10973.548164,10640.323215


In [113]:
# Hallamos los precios que maximicen la rentabilidad en cada segmento
max_ren_seg0 = df_elasticidad.loc[(df_elasticidad['rentabilidad_seg0'] == df_elasticidad['rentabilidad_seg0'].max())]
max_ren_seg1 = df_elasticidad.loc[(df_elasticidad['rentabilidad_seg1'] == df_elasticidad['rentabilidad_seg1'].max())]
max_ren_seg2 = df_elasticidad.loc[(df_elasticidad['rentabilidad_seg2'] == df_elasticidad['rentabilidad_seg2'].max())]
max_ren_seg3 = df_elasticidad.loc[(df_elasticidad['rentabilidad_seg3'] == df_elasticidad['rentabilidad_seg3'].max())]
max_ren_seg4 = df_elasticidad.loc[(df_elasticidad['rentabilidad_seg4'] == df_elasticidad['rentabilidad_seg4'].max())]

In [114]:
max_ren_seg0

,irr,prob_seg0,prob_seg1,prob_seg2,prob_seg3,prob_seg4,requested_amount,emi_amount,cibil_score,loan_term,...,gastos_seg0,gastos_seg1,gastos_seg2,gastos_seg3,gastos_seg4,rentabilidad_seg0,rentabilidad_seg1,rentabilidad_seg2,rentabilidad_seg3,rentabilidad_seg4
5,17.0526,0.82549,0.850295,0.798667,0.791147,0.765782,400000.0,12500.0,775.0,36.0,...,29717.642341,30610.611384,28752.000131,28481.304075,27568.161769,12512.613307,12888.597923,12106.029655,11992.05308,11607.574512


In [115]:
max_ren_seg1

,irr,prob_seg0,prob_seg1,prob_seg2,prob_seg3,prob_seg4,requested_amount,emi_amount,cibil_score,loan_term,...,gastos_seg0,gastos_seg1,gastos_seg2,gastos_seg3,gastos_seg4,rentabilidad_seg0,rentabilidad_seg1,rentabilidad_seg2,rentabilidad_seg3,rentabilidad_seg4
7,17.4737,0.75474,0.79495,0.736029,0.718011,0.700671,400000.0,12300.0,774.0,36.0,...,27170.639712,28618.215279,26497.042555,25848.400637,25224.144553,12393.660882,13053.960414,12086.405152,11790.532547,11505.783336


In [116]:
fig = go.Figure()

# Se agrega el precio máximo de ingreso
fig.add_trace(go.Scatter(x=df_elasticidad['irr'], y=df_elasticidad['ingresos_seg0'], name='Ingresos ($)'))
# Se agrega la etiqueta
fig.add_annotation(x=float(max_ing_seg0['irr']), y=float(max_ing_seg0['ingresos_seg0']),
                                  text="MaxPrecioIng:"+str(float(max_ing_seg0['irr'].round(2))),
                                  showarrow=True,
                                  arrowhead=1)
# Se agrega la recta del precio máximo
fig.add_vline(x=float(max_ing_seg0['irr']), line_width=2, line_dash="dash",
                             line_color="blue", opacity=0.25)

# Se agrega el precio máximo de rentabilidad
fig.add_trace(go.Scatter(x=df_elasticidad['irr'], y=df_elasticidad['rentabilidad_seg0'], name='Rentabilidad ($)'))
# Se agrega la etiqueta
fig.add_annotation(x=float(max_ren_seg0['irr']), y=float(max_ren_seg0['rentabilidad_seg0']),
                                  text="MaxPrecioRent:"+str(float(max_ren_seg0['irr'].round(2))),
                                  showarrow=True,
                                  arrowhead=1)
# Se agrega la recta del precio máximo
fig.add_vline(x=float(max_ren_seg0['irr']), line_width=2, line_dash="dash",
                             line_color="green", opacity=0.25)

# # Agregamos la línea de restricción: Costo
# fig.add_vline(x=float(df_elasticidad['costo'].mean()), line_width=2, line_dash="dash",
#                              line_color="orange", opacity=0.5)

# Configuración del gráfico
fig.update_layout(
    showlegend=True,
    width=800,
    height=600,
    xaxis_title="Precio ($)",
    yaxis_title="Importe ($)",
    title="Precios óptimos en Segmento 0")


In [117]:
fig = go.Figure()

# Se agrega el precio máximo de ingreso
fig.add_trace(go.Scatter(x=df_elasticidad['irr'], y=df_elasticidad['ingresos_seg1'], name='Ingresos ($)'))
# Se agrega la etiqueta
fig.add_annotation(x=float(max_ing_seg1['irr']), y=float(max_ing_seg1['ingresos_seg1']),
                                  text="MaxPrecioIng:"+str(float(max_ing_seg1['irr'].round(2))),
                                  showarrow=True,
                                  arrowhead=1)
# Se agrega la recta del precio máximo
fig.add_vline(x=float(max_ing_seg1['irr']), line_width=2, line_dash="dash",
                             line_color="blue", opacity=0.25)

# Se agrega el precio máximo de rentabilidad
fig.add_trace(go.Scatter(x=df_elasticidad['irr'], y=df_elasticidad['rentabilidad_seg1'], name='Rentabilidad ($)'))
# Se agrega la etiqueta
fig.add_annotation(x=float(max_ren_seg1['irr']), y=float(max_ren_seg1['rentabilidad_seg1']),
                                  text="MaxPrecioRent:"+str(float(max_ren_seg1['irr'].round(2))),
                                  showarrow=True,
                                  arrowhead=1)
# Se agrega la recta del precio máximo
fig.add_vline(x=float(max_ren_seg1['irr']), line_width=2, line_dash="dash",
                             line_color="green", opacity=0.25)

# # Agregamos la línea de restricción: Costo
# fig.add_vline(x=float(df_elasticidad['costo'].mean()), line_width=2, line_dash="dash",
#                              line_color="orange", opacity=0.5)

# Configuración del gráfico
fig.update_layout(
    showlegend=True,
    width=800,
    height=600,
    xaxis_title="Precio ($)",
    yaxis_title="Importe ($)",
    title="Precios óptimos en Segmento 1")

In [118]:
fig = go.Figure()

# Se agrega el precio máximo de ingreso
fig.add_trace(go.Scatter(x=df_elasticidad['irr'], y=df_elasticidad['ingresos_seg2'], name='Ingresos ($)'))
# Se agrega la etiqueta
fig.add_annotation(x=float(max_ing_seg2['irr']), y=float(max_ing_seg2['ingresos_seg2']),
                                  text="MaxPrecioIng:"+str(float(max_ing_seg2['irr'].round(2))),
                                  showarrow=True,
                                  arrowhead=1)
# Se agrega la recta del precio máximo
fig.add_vline(x=float(max_ing_seg2['irr']), line_width=2, line_dash="dash",
                             line_color="blue", opacity=0.25)

# Se agrega el precio máximo de rentabilidad
fig.add_trace(go.Scatter(x=df_elasticidad['irr'], y=df_elasticidad['rentabilidad_seg2'], name='Rentabilidad ($)'))
# Se agrega la etiqueta
fig.add_annotation(x=float(max_ren_seg2['irr']), y=float(max_ren_seg2['rentabilidad_seg2']),
                                  text="MaxPrecioRent:"+str(float(max_ren_seg2['irr'].round(2))),
                                  showarrow=True,
                                  arrowhead=1)
# Se agrega la recta del precio máximo
fig.add_vline(x=float(max_ren_seg2['irr']), line_width=2, line_dash="dash",
                             line_color="green", opacity=0.25)

# # Agregamos la línea de restricción: Costo
# fig.add_vline(x=float(df_elasticidad['costo'].mean()), line_width=2, line_dash="dash",
#                              line_color="orange", opacity=0.5)

# Configuración del gráfico
fig.update_layout(
    showlegend=True,
    width=800,
    height=600,
    xaxis_title="Precio ($)",
    yaxis_title="Importe ($)",
    title="Precios óptimos en Segmento 2")

In [119]:
fig = go.Figure()

# Se agrega el precio máximo de ingreso
fig.add_trace(go.Scatter(x=df_elasticidad['irr'], y=df_elasticidad['ingresos_seg3'], name='Ingresos ($)'))
# Se agrega la etiqueta
fig.add_annotation(x=float(max_ing_seg3['irr']), y=float(max_ing_seg3['ingresos_seg3']),
                                  text="MaxPrecioIng:"+str(float(max_ing_seg3['irr'].round(2))),
                                  showarrow=True,
                                  arrowhead=1)
# Se agrega la recta del precio máximo
fig.add_vline(x=float(max_ing_seg3['irr']), line_width=2, line_dash="dash",
                             line_color="blue", opacity=0.25)

# Se agrega el precio máximo de rentabilidad
fig.add_trace(go.Scatter(x=df_elasticidad['irr'], y=df_elasticidad['rentabilidad_seg3'], name='Rentabilidad ($)'))
# Se agrega la etiqueta
fig.add_annotation(x=float(max_ren_seg3['irr']), y=float(max_ren_seg3['rentabilidad_seg3']),
                                  text="MaxPrecioRent:"+str(float(max_ren_seg3['irr'].round(2))),
                                  showarrow=True,
                                  arrowhead=1)
# Se agrega la recta del precio máximo
fig.add_vline(x=float(max_ren_seg3['irr']), line_width=2, line_dash="dash",
                             line_color="green", opacity=0.25)

# # Agregamos la línea de restricción: Costo
# fig.add_vline(x=float(df_elasticidad['costo'].mean()), line_width=2, line_dash="dash",
#                              line_color="orange", opacity=0.5)

# Configuración del gráfico
fig.update_layout(
    showlegend=True,
    width=800,
    height=600,
    xaxis_title="Precio ($)",
    yaxis_title="Importe ($)",
    title="Precios óptimos en Segmento 3")

In [120]:
fig = go.Figure()

# Se agrega el precio máximo de ingreso
fig.add_trace(go.Scatter(x=df_elasticidad['irr'], y=df_elasticidad['ingresos_seg4'], name='Ingresos ($)'))
# Se agrega la etiqueta
fig.add_annotation(x=float(max_ing_seg3['irr']), y=float(max_ing_seg3['ingresos_seg4']),
                                  text="MaxPrecioIng:"+str(float(max_ing_seg3['irr'].round(2))),
                                  showarrow=True,
                                  arrowhead=1)
# Se agrega la recta del precio máximo
fig.add_vline(x=float(max_ing_seg3['irr']), line_width=2, line_dash="dash",
                             line_color="blue", opacity=0.25)

# Se agrega el precio máximo de rentabilidad
fig.add_trace(go.Scatter(x=df_elasticidad['irr'], y=df_elasticidad['rentabilidad_seg4'], name='Rentabilidad ($)'))
# Se agrega la etiqueta
fig.add_annotation(x=float(max_ren_seg3['irr']), y=float(max_ren_seg3['rentabilidad_seg4']),
                                  text="MaxPrecioRent:"+str(float(max_ren_seg3['irr'].round(2))),
                                  showarrow=True,
                                  arrowhead=1)
# Se agrega la recta del precio máximo
fig.add_vline(x=float(max_ren_seg3['irr']), line_width=2, line_dash="dash",
                             line_color="green", opacity=0.25)

# # Agregamos la línea de restricción: Costo
# fig.add_vline(x=float(df_elasticidad['costo'].mean()), line_width=2, line_dash="dash",
#                              line_color="orange", opacity=0.5)

# Configuración del gráfico
fig.update_layout(
    showlegend=True,
    width=800,
    height=600,
    xaxis_title="Precio ($)",
    yaxis_title="Importe ($)",
    title="Precios óptimos en Segmento 4")

In [121]:
fig = go.Figure()

# Se agrega el precio máximo de ingreso
fig.add_trace(go.Scatter(x=df_elasticidad['ingresos_seg0'], y=df_elasticidad['rentabilidad_seg0'], name='Frontera Eficiente'))

# Se agrega la etiqueta de ingreso
fig.add_annotation(x=float(max_ing_seg0['ingresos_seg0']), y=float(max_ing_seg0['rentabilidad_seg0']),
                                  text="MaxPrecioIng:"+str(float(max_ing_seg0['irr'].round(2))),
                                  showarrow=True,
                                  arrowhead=1)
# Se agrega la recta del precio máximo
fig.add_vline(x=float(max_ing_seg0['ingresos_seg0']), line_width=2, line_dash="dash",
                             line_color="blue", opacity=0.25)

# Se agrega la etiqueta de rentabilidad
fig.add_annotation(x=float(max_ren_seg0['ingresos_seg0']), y=float(max_ren_seg0['rentabilidad_seg0']),
                                  text="MaxPrecioRent:"+str(float(max_ren_seg0['irr'].round(2))),
                                  showarrow=True,
                                  arrowhead=1)
# Se agrega la recta del precio máximo
fig.add_vline(x=float(max_ren_seg0['ingresos_seg0']), line_width=2, line_dash="dash",
                             line_color="green", opacity=0.25)

# Configuración del gráfico
fig.update_layout(
    showlegend=False,
    width=800,
    height=600,
    xaxis=dict(
        title="Ingreso ($)"
       ## ,range=[1, 2]
        ),
   yaxis=dict(
        title="Rentabilidad ($)"
       ##,range=[-0.5, 1]
    ),
    title="Frontera Eficiente Segmento 0")

In [122]:
fig = go.Figure()

# Se agrega el precio máximo de ingreso
fig.add_trace(go.Scatter(x=df_elasticidad['ingresos_seg1'], y=df_elasticidad['rentabilidad_seg1'], name='Frontera Eficiente'))

# Se agrega la etiqueta de ingreso
fig.add_annotation(x=float(max_ing_seg1['ingresos_seg1']), y=float(max_ing_seg1['rentabilidad_seg1']),
                                  text="MaxPrecioIng:"+str(float(max_ing_seg1['irr'].round(2))),
                                  showarrow=True,
                                  arrowhead=1)
# Se agrega la recta del precio máximo
fig.add_vline(x=float(max_ing_seg1['ingresos_seg1']), line_width=2, line_dash="dash",
                             line_color="blue", opacity=0.25)

# Se agrega la etiqueta de rentabilidad
fig.add_annotation(x=float(max_ren_seg1['ingresos_seg1']), y=float(max_ren_seg1['rentabilidad_seg1']),
                                  text="MaxPrecioRent:"+str(float(max_ren_seg1['irr'].round(2))),
                                  showarrow=True,
                                  arrowhead=1)
# Se agrega la recta del precio máximo
fig.add_vline(x=float(max_ren_seg1['ingresos_seg1']), line_width=2, line_dash="dash",
                             line_color="green", opacity=0.25)

# Configuración del gráfico
fig.update_layout(
    showlegend=False,
    width=800,
    height=600,
     xaxis=dict(
        title="Ingreso ($)"
      ##  ,range=[1.5, 2.5]
        ),
      yaxis=dict(
        title="Rentabilidad ($)"
       ##,range=[-0.5, 1]
      ),
    title="Frontera Eficiente Segmento 1")

In [123]:
fig = go.Figure()

# Se agrega el precio máximo de ingreso
fig.add_trace(go.Scatter(x=df_elasticidad['ingresos_seg2'], y=df_elasticidad['rentabilidad_seg2'], name='Frontera Eficiente'))

# Se agrega la etiqueta de ingreso
fig.add_annotation(x=float(max_ing_seg2['ingresos_seg2']), y=float(max_ing_seg2['rentabilidad_seg2']),
                                  text="MaxPrecioIng:"+str(float(max_ing_seg2['irr'].round(2))),
                                  showarrow=True,
                                  arrowhead=1)
# Se agrega la recta del precio máximo
fig.add_vline(x=float(max_ing_seg2['ingresos_seg2']), line_width=2, line_dash="dash",
                             line_color="blue", opacity=0.25)

# Se agrega la etiqueta de rentabilidad
fig.add_annotation(x=float(max_ren_seg2['ingresos_seg2']), y=float(max_ren_seg2['rentabilidad_seg2']),
                                  text="MaxPrecioRent:"+str(float(max_ren_seg2['irr'].round(2))),
                                  showarrow=True,
                                  arrowhead=1)
# Se agrega la recta del precio máximo
fig.add_vline(x=float(max_ren_seg2['ingresos_seg2']), line_width=2, line_dash="dash",
                             line_color="green", opacity=0.25)

# Configuración del gráfico
fig.update_layout(
    showlegend=False,
    width=800,
    height=600,
    xaxis_title="Ingreso ($)",
    yaxis_title="Rentabilidad ($)",
    title="Frontera Eficiente Segmento 2")

In [124]:
fig = go.Figure()

# Se agrega el precio máximo de ingreso
fig.add_trace(go.Scatter(x=df_elasticidad['ingresos_seg3'], y=df_elasticidad['rentabilidad_seg3'], name='Frontera Eficiente'))

# Se agrega la etiqueta de ingreso
fig.add_annotation(x=float(max_ing_seg3['ingresos_seg3']), y=float(max_ing_seg3['rentabilidad_seg3']),
                                  text="MaxPrecioIng:"+str(float(max_ing_seg3['irr'].round(2))),
                                  showarrow=True,
                                  arrowhead=1)
# Se agrega la recta del precio máximo
fig.add_vline(x=float(max_ing_seg3['ingresos_seg3']), line_width=2, line_dash="dash",
                             line_color="blue", opacity=0.25)

# Se agrega la etiqueta de rentabilidad
fig.add_annotation(x=float(max_ren_seg3['ingresos_seg3']), y=float(max_ren_seg3['rentabilidad_seg3']),
                                  text="MaxPrecioRent:"+str(float(max_ren_seg3['irr'].round(2))),
                                  showarrow=True,
                                  arrowhead=1)
# Se agrega la recta del precio máximo
fig.add_vline(x=float(max_ren_seg3['ingresos_seg3']), line_width=2, line_dash="dash",
                             line_color="green", opacity=0.25)

# Configuración del gráfico
fig.update_layout(
    showlegend=False,
    width=800,
    height=600,
    xaxis_title="Ingreso ($)",
    yaxis_title="Rentabilidad ($)",
    title="Frontera Eficiente Segmento 3")

In [125]:
fig = go.Figure()

# Se agrega el precio máximo de ingreso
fig.add_trace(go.Scatter(x=df_elasticidad['ingresos_seg4'], y=df_elasticidad['rentabilidad_seg4'], name='Frontera Eficiente'))

# Se agrega la etiqueta de ingreso
fig.add_annotation(x=float(max_ing_seg4['ingresos_seg4']), y=float(max_ing_seg4['rentabilidad_seg4']),
                                  text="MaxPrecioIng:"+str(float(max_ing_seg4['irr'].round(2))),
                                  showarrow=True,
                                  arrowhead=1)
# Se agrega la recta del precio máximo
fig.add_vline(x=float(max_ing_seg4['ingresos_seg4']), line_width=2, line_dash="dash",
                             line_color="blue", opacity=0.25)

# Se agrega la etiqueta de rentabilidad
fig.add_annotation(x=float(max_ren_seg4['ingresos_seg4']), y=float(max_ren_seg4['rentabilidad_seg4']),
                                  text="MaxPrecioRent:"+str(float(max_ren_seg4['irr'].round(2))),
                                  showarrow=True,
                                  arrowhead=1)
# Se agrega la recta del precio máximo
fig.add_vline(x=float(max_ren_seg4['ingresos_seg4']), line_width=2, line_dash="dash",
                             line_color="green", opacity=0.25)

# Configuración del gráfico
fig.update_layout(
    showlegend=False,
    width=800,
    height=600,
    xaxis_title="Ingreso ($)",
    yaxis_title="Rentabilidad ($)",
    title="Frontera Eficiente Segmento 4")